<a href="https://colab.research.google.com/github/RehabEmam228/Bertlesmann-challenge/blob/master/Sentiment_RNN_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis with an RNN

In this notebook, you'll implement a recurrent neural network that performs sentiment analysis. 
>Using an RNN rather than a strictly feedforward network is more accurate since we can include information about the *sequence* of words. 

Here we'll use a dataset of movie reviews, accompanied by sentiment labels: positive or negative.

<img src="https://github.com/simplesolutionssas/UdacityDeepLearningNanodegree/blob/master/9.%20sentiment-rnn/assets/reviews_ex.png?raw=1" width=40%>

### Network Architecture

The architecture for this network is shown below.

<img src="https://github.com/simplesolutionssas/UdacityDeepLearningNanodegree/blob/master/9.%20sentiment-rnn/assets/network_diagram.png?raw=1" width=40%>

>**First, we'll pass in words to an embedding layer.** We need an embedding layer because we have tens of thousands of words, so we'll need a more efficient representation for our input data than one-hot encoded vectors. You should have seen this before from the Word2Vec lesson. You can actually train an embedding with the Skip-gram Word2Vec model and use those embeddings as input, here. However, it's good enough to just have an embedding layer and let the network learn a different embedding table on its own. *In this case, the embedding layer is for dimensionality reduction, rather than for learning semantic representations.*

>**After input words are passed to an embedding layer, the new embeddings will be passed to LSTM cells.** The LSTM cells will add *recurrent* connections to the network and give us the ability to include information about the *sequence* of words in the movie review data. 

>**Finally, the LSTM outputs will go to a sigmoid output layer.** We're using a sigmoid function because positive and negative = 1 and 0, respectively, and a sigmoid will output predicted, sentiment values between 0-1. 

We don't care about the sigmoid outputs except for the **very last one**; we can ignore the rest. We'll calculate the loss by comparing the output at the last time step and the training label (pos or neg).

## Importing libraries and define training mode

In [0]:
import numpy as np
from string import punctuation
from collections import Counter
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import itertools

train_on_gpu = torch.cuda.is_available()

## Loading and visualizing the data

In [0]:
# download the files locally
!wget -c -nc https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/master/sentiment-rnn/data/reviews.txt
!wget -c -nc https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/master/sentiment-rnn/data/labels.txt

# read data from text files
with open('reviews.txt', 'r') as f:
    reviews = f.read()
with open('labels.txt', 'r') as f:
    labels = f.read()

print(reviews[:2000])
print()
print(labels[:20])

File ‘reviews.txt’ already there; not retrieving.

File ‘labels.txt’ already there; not retrieving.

bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   
story of a man who has unnatural feelings for a pig . starts out w


## Pre-processing the data (reviews and labels)

In [0]:
def preprocess_labels(labels): 
  # 1=positive, 0=negative label conversion
  labels_split = labels.split('\n')
  encoded_labels = [1 if label == 'positive' else 0 for label in labels_split]
  
  return encoded_labels
  

def remove_outliers(encoded_reviews, encoded_labels):
  for idx, review in enumerate(encoded_reviews):
    if len(review) == 0:
      del encoded_reviews[idx]
      del encoded_labels[idx]
    
  return encoded_reviews, encoded_labels


def pad_reviews(encoded_reviews, seq_length):
  for idx, review in enumerate(encoded_reviews):
    word_count = len(review)
    if word_count < seq_length:
      # add as many zeros as needed at the beginning, and then append the review.
      encoded_reviews[idx] = ([0] * (seq_length - word_count)) + review
    else: 
      # truncate the review to just seq_length words
      encoded_reviews[idx] = review[:seq_length]

  # return the encoded_reviews as a Numpy array, which is the expected format.
  return encoded_reviews


def preprocess_data(reviews, labels, sequence_length=200): 
  # convert to lower case, remove special characters, remove line breaks. 
  lower_reviews = reviews.lower()
  clean_reviews = ''.join([c for c in lower_reviews if c not in punctuation])
  split_reviews = clean_reviews.split('\n')
  joined_reviews = ' '.join(split_reviews)
  # create a list with all the words contained in the reviews.  
  word_list = joined_reviews.split()
  # produce a dictionary to encode words (convert from words to integers).
  counts = Counter(word_list)
  vocab = sorted(counts, key=counts.get, reverse=True)
  vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}

  # produce the encoded reviews and labels. 
  encoded_reviews = [[vocab_to_int[vocab] for vocab in review.split()] for review in split_reviews]
  encoded_labels = preprocess_labels(labels)
  # remove zero length reviews (with the respective labels). 
  encoded_reviews_wo_outliers, encoded_labels_wo_outliers = remove_outliers(encoded_reviews, encoded_labels)
  # pad the resulting reviews. 
  padded_reviews = pad_reviews(encoded_reviews_wo_outliers, sequence_length)
  # convert to numpy arrays. 
  preprocessed_reviews, preprocessed_labels = np.asarray(padded_reviews), np.asarray(encoded_labels_wo_outliers)

  return preprocessed_reviews, preprocessed_labels, vocab_to_int

## Generating the training, validation and test sets

In [0]:
def create_data_loaders(reviews, labels, sequence_length=200, batch_size=50, training_fraction=0.8, validation_fraction=0.1, shuffle_data=True):
  # first, clean and pre-process the data. 
  features, labels, vocab_to_int = preprocess_data(reviews, labels, sequence_length)

  # determine each data set size. 
  full_set_size = len(features)
  training_set_size = np.int(full_set_size * training_fraction)
  validation_set_size = np.int(full_set_size * validation_fraction)
  test_set_size = full_set_size - training_set_size - validation_set_size

  ## get the data for the training set (features and labels, x and y)
  training_indexes = np.arange(training_set_size)
  training_data, training_labels = features[training_indexes], labels[training_indexes]
  # now get the data for the validation set. 
  validation_set_start, validation_set_end = training_set_size, training_set_size + validation_set_size
  validation_indexes = np.arange(validation_set_start, validation_set_end)
  validation_data, validation_labels = features[validation_indexes], labels[validation_indexes]
  # finally get the data for the test set. 
  test_set_start, test_set_end = validation_set_end, validation_set_end + test_set_size
  test_indexes = np.arange(test_set_start, test_set_end)
  test_data, test_labels = features[test_indexes], labels[test_indexes]

  # create the tensor datasets required to create the data loaders.
  train_data = TensorDataset(torch.from_numpy(training_data), torch.from_numpy(training_labels))
  valid_data = TensorDataset(torch.from_numpy(validation_data), torch.from_numpy(validation_labels))
  test_data = TensorDataset(torch.from_numpy(test_data), torch.from_numpy(test_labels))  

  # finally, create the data loaders. 
  train_loader = DataLoader(train_data, shuffle=shuffle_data, batch_size=batch_size)
  valid_loader = DataLoader(valid_data, shuffle=shuffle_data, batch_size=batch_size)
  test_loader = DataLoader(test_data, shuffle=shuffle_data, batch_size=batch_size)
  
  return train_loader, valid_loader, test_loader, vocab_to_int

## Defining the Sentiment Network with PyTorch

In [0]:
class SentimentRNN(nn.Module):

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, num_layers, drop_prob=0.5):
        super(SentimentRNN, self).__init__()

        self.output_size = output_size
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim
        
        # define all layers
        self.embed = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, dropout=drop_prob, batch_first=True)
        self.drop = nn.Dropout(drop_prob) 
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sigmoid = nn.Sigmoid()
        

    def forward(self, x, hidden):
        inputs = self.embed(x)
        out, hidden = self.lstm(inputs, hidden)
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.drop(out)
        out = self.fc(out)
        sig_out = self.sigmoid(out)

        # reshape to the batch size first
        batch_size = x.size()
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1]

        # return last sigmoid output and hidden state
        return sig_out, hidden
    
    
    def init_hidden(self, batch_size):
        # create two new tensors with sizes num_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data

        if (train_on_gpu):
          hidden = (weight.new(self.num_layers, batch_size, self.hidden_dim).zero_().cuda(),
                    weight.new(self.num_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
          hidden = (weight.new(self.num_layers, batch_size, self.hidden_dim).zero_(),
                    weight.new(self.num_layers, batch_size, self.hidden_dim).zero_())
          
        return hidden

---
## Training

In [0]:
def train(net, training_loader, validation_loader, batch_size, criterion, optimizer, epochs):
  
  # training parameters
  print_every = 100
  clip = 5 # gradient clipping
  counter = 0

  if train_on_gpu: net.cuda()     
  net.train()
  # train for some number of epochs
  for e in range(epochs):
      # initialize hidden state
      h = net.init_hidden(batch_size)

      # batch loop
      for inputs, labels in training_loader:
          counter += 1
          if train_on_gpu: inputs, labels = inputs.cuda(), labels.cuda()
          # creating new variables for the hidden state, otherwise 
          # we'd backprop through the entire training history
          h = tuple([each.data for each in h])
          # zero accumulated gradients
          net.zero_grad()
          # get the output from the model
          output, h = net(inputs, h)
          # calculate the loss and perform backprop
          loss = criterion(output.squeeze(), labels.float())
          loss.backward()
          # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
          nn.utils.clip_grad_norm_(net.parameters(), clip)
          optimizer.step()

          # loss stats
          if counter % print_every == 0:
              # Get validation loss
              val_h = net.init_hidden(batch_size)
              val_losses = []
              net.eval()
              for inputs, labels in validation_loader:
                  # Creating new variables for the hidden state, otherwise
                  # we'd backprop through the entire training history
                  val_h = tuple([each.data for each in val_h])
                  if train_on_gpu: inputs, labels = inputs.cuda(), labels.cuda()
                  output, val_h = net(inputs, val_h)
                  val_loss = criterion(output.squeeze(), labels.float())
                  val_losses.append(val_loss.item())

              net.train()
              print("Epoch: {}/{}...".format(e+1, epochs),
                    "Step: {}...".format(counter),
                    "Loss: {:.6f}...".format(loss.item()),
                    "Val Loss: {:.6f}".format(np.mean(val_losses)))
              
  return net

---
## Testing

In [0]:
 def test(net, test_loader, batch_size, criterion):
  # Get test data loss and accuracy
  test_losses = [] # track loss
  num_correct = 0
  # init hidden state
  h = net.init_hidden(batch_size)
  net.eval()
  # iterate over test data
  for inputs, labels in test_loader:
      # Creating new variables for the hidden state, otherwise
      # we'd backprop through the entire training history
      h = tuple([each.data for each in h])
      if train_on_gpu: inputs, labels = inputs.cuda(), labels.cuda()
      # get predicted outputs
      output, h = net(inputs, h)
      # calculate loss
      test_loss = criterion(output.squeeze(), labels.float())
      test_losses.append(test_loss.item())
      # convert output probabilities to predicted class (0 or 1)
      pred = torch.round(output.squeeze())  # rounds to the nearest integer
      # compare predictions to true label
      correct_tensor = pred.eq(labels.float().view_as(pred))
      correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
      num_correct += np.sum(correct)

  # -- stats! -- ##
  # avg test loss
  print("Test loss: {:.3f}".format(np.mean(test_losses)))

  # accuracy over all test data
  test_accuracy = num_correct/len(test_loader.dataset)
  print("Test accuracy: {:.3f}".format(test_accuracy))

  return test_accuracy

## Saving and loading the model

In [0]:
def save_checkpoint(filepath, net, vocab_size, output_size, embedding_dim, hidden_dim, num_layers, dropout):
  checkpoint = {'vocab_size': vocab_size,
                'output_size': output_size,
                'embedding_dim': embedding_dim,
                'hidden_dim': hidden_dim,
                'num_layers': num_layers,
                'drop_prob': dropout,
                'state_dict': net.state_dict()}

  torch.save(checkpoint, filepath)


def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    net = SentimentRNN(checkpoint['vocab_size'],
                       checkpoint['output_size'],
                       checkpoint['embedding_dim'],
                       checkpoint['hidden_dim'],
                       checkpoint['num_layers'],
                       checkpoint['drop_prob'])
    net.load_state_dict(checkpoint['state_dict'])
    
    return net

# Best model selection

In [0]:
# get the required data for training, validating and testing the model. 
sequence_length=200
batch_size = 50
training_fraction=0.8
validation_fraction=0.1
training_loader, validation_loader, test_loader, vocab_to_int = create_data_loaders(reviews, labels, sequence_length, batch_size, training_fraction, validation_fraction, False)

# fixed model hyperparams
vocab_size = len(vocab_to_int) + 1 # +1 for the 0 padding + word token count
output_size = 1
learning_rate=0.001
epochs = 6 # 3-4 is approx where I noticed the validation loss stop decreasing

# variable model hyperparams
embedding_dims = [100, 200, 400] # Number of columns in the embedding lookup table; size of our embeddings.
hidden_dims = [64, 128, 256] # Number of units in the hidden layers of our LSTM cells. Usually larger is better performance wise. Common values are 128, 256, 512, etc.
nums_layers = [2, 3, 4] # Number of LSTM layers in the network. Typically between 1-3
dropouts = [0.1, 0.2, 0.3]

best_accuracy = 0
experiment = 1
# run the different experiments and save the best one. 
for (embedding_dim, hidden_dim, num_layers, dropout) in list(itertools.product(embedding_dims, hidden_dims, nums_layers, dropouts)):
  print('\n\n## Experiment {} -'.format(experiment),
        'embedding_dim', embedding_dim,
        ', hidden_dim', hidden_dim,
        ', num_layers', num_layers, 
        ', dropout', dropout)
  net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, num_layers, dropout)
  criterion = nn.BCELoss()
  optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
  net = train(net, training_loader, validation_loader, batch_size, criterion, optimizer, epochs)
  accuracy = test(net, test_loader, batch_size, criterion)
  if accuracy > best_accuracy:
    print('New best model saved:', net)
    save_checkpoint('Best_RNN.pth', net, vocab_size, output_size, embedding_dim, hidden_dim, num_layers, dropout)
    best_accuracy = accuracy
  
  print('## End of experiment # ', experiment)
  experiment += 1



## Experiment 1 - embedding_dim 100 , hidden_dim 64 , num_layers 1 , dropout 0.1


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Epoch: 1/6... Step: 100... Loss: 0.690882... Val Loss: 0.688793
Epoch: 1/6... Step: 200... Loss: 0.913858... Val Loss: 0.899843
Epoch: 1/6... Step: 300... Loss: 0.686175... Val Loss: 0.686361
Epoch: 1/6... Step: 400... Loss: 0.686980... Val Loss: 0.666352
Epoch: 2/6... Step: 500... Loss: 0.593717... Val Loss: 0.620114
Epoch: 2/6... Step: 600... Loss: 0.586512... Val Loss: 0.602823
Epoch: 2/6... Step: 700... Loss: 0.628718... Val Loss: 0.571068
Epoch: 2/6... Step: 800... Loss: 0.615742... Val Loss: 0.565879
Epoch: 3/6... Step: 900... Loss: 0.571425... Val Loss: 0.610948
Epoch: 3/6... Step: 1000... Loss: 0.439332... Val Loss: 0.521452
Epoch: 3/6... Step: 1100... Loss: 0.588382... Val Loss: 0.590028
Epoch: 3/6... Step: 1200... Loss: 0.536237... Val Loss: 0.506189
Epoch: 4/6... Step: 1300... Loss: 0.558006... Val Loss: 0.586604
Epoch: 4/6... Step: 1400... Loss: 0.390470... Val Loss: 0.495036
Epoch: 4/6... Step: 1500... Loss: 0.493868... Val Loss: 0.529131
Epoch: 4/6... Step: 1600... Loss: 

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Epoch: 1/6... Step: 100... Loss: 0.686370... Val Loss: 0.689323
Epoch: 1/6... Step: 200... Loss: 0.681751... Val Loss: 0.629485
Epoch: 1/6... Step: 300... Loss: 0.718304... Val Loss: 0.659737
Epoch: 1/6... Step: 400... Loss: 0.652282... Val Loss: 0.635140
Epoch: 2/6... Step: 500... Loss: 0.692675... Val Loss: 0.659288
Epoch: 2/6... Step: 600... Loss: 0.496302... Val Loss: 0.620707
Epoch: 2/6... Step: 700... Loss: 0.679939... Val Loss: 0.572985
Epoch: 2/6... Step: 800... Loss: 0.378180... Val Loss: 0.560410
Epoch: 3/6... Step: 900... Loss: 0.572255... Val Loss: 0.550661
Epoch: 3/6... Step: 1000... Loss: 0.573366... Val Loss: 0.575445
Epoch: 3/6... Step: 1100... Loss: 0.628177... Val Loss: 0.561852
Epoch: 3/6... Step: 1200... Loss: 0.363245... Val Loss: 0.538020
Epoch: 4/6... Step: 1300... Loss: 0.534582... Val Loss: 0.545656
Epoch: 4/6... Step: 1400... Loss: 0.404250... Val Loss: 0.520840
Epoch: 4/6... Step: 1500... Loss: 0.399702... Val Loss: 0.470162
Epoch: 4/6... Step: 1600... Loss: 

...

Epoch: 1/6... Step: 100... Loss: 0.697035... Val Loss: 0.689276
Epoch: 1/6... Step: 200... Loss: 0.719386... Val Loss: 0.781002
Epoch: 1/6... Step: 300... Loss: 0.695107... Val Loss: 0.615675
Epoch: 1/6... Step: 400... Loss: 0.505158... Val Loss: 0.647428
Epoch: 2/6... Step: 500... Loss: 0.673053... Val Loss: 0.655217
Epoch: 2/6... Step: 600... Loss: 0.510416... Val Loss: 0.575194
Epoch: 2/6... Step: 700... Loss: 0.616547... Val Loss: 0.578644
Epoch: 2/6... Step: 800... Loss: 0.699186... Val Loss: 0.641472
Epoch: 3/6... Step: 900... Loss: 0.541888... Val Loss: 0.558097
Epoch: 3/6... Step: 1000... Loss: 0.452913... Val Loss: 0.516253
Epoch: 3/6... Step: 1100... Loss: 0.528013... Val Loss: 0.522319
Epoch: 3/6... Step: 1200... Loss: 0.598583... Val Loss: 0.521264
Epoch: 4/6... Step: 1300... Loss: 0.551102... Val Loss: 0.578798
Epoch: 4/6... Step: 1400... Loss: 0.252899... Val Loss: 0.471118
Epoch: 4/6... Step: 1500... Loss: 0.383418... Val Loss: 0.490756
Epoch: 4/6... Step: 1600... Loss: 

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.4 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Epoch: 1/6... Step: 100... Loss: 0.686281... Val Loss: 0.688223
Epoch: 1/6... Step: 200... Loss: 0.744930... Val Loss: 0.707575
Epoch: 1/6... Step: 300... Loss: 0.782716... Val Loss: 0.682914
Epoch: 1/6... Step: 400... Loss: 0.668225... Val Loss: 0.627781
Epoch: 2/6... Step: 500... Loss: 0.564455... Val Loss: 0.581168
Epoch: 2/6... Step: 600... Loss: 0.651703... Val Loss: 0.578745
Epoch: 2/6... Step: 700... Loss: 0.648441... Val Loss: 0.579017
Epoch: 2/6... Step: 800... Loss: 0.660783... Val Loss: 0.617757
Epoch: 3/6... Step: 900... Loss: 0.428126... Val Loss: 0.537604
Epoch: 3/6... Step: 1000... Loss: 0.495213... Val Loss: 0.573015
Epoch: 3/6... Step: 1100... Loss: 0.561780... Val Loss: 0.519751
Epoch: 3/6... Step: 1200... Loss: 0.391529... Val Loss: 0.527537
Epoch: 4/6... Step: 1300... Loss: 0.392433... Val Loss: 0.492816
Epoch: 4/6... Step: 1400... Loss: 0.050559... Val Loss: 0.347309
Epoch: 4/6... Step: 1500... Loss: 0.521859... Val Loss: 0.509922
Epoch: 4/6... Step: 1600... Loss: 

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Epoch: 1/6... Step: 100... Loss: 0.671290... Val Loss: 0.689422
Epoch: 1/6... Step: 200... Loss: 0.701253... Val Loss: 0.661480
Epoch: 1/6... Step: 300... Loss: 0.464537... Val Loss: 1.181636
Epoch: 1/6... Step: 400... Loss: 0.491654... Val Loss: 1.111220
Epoch: 2/6... Step: 500... Loss: 0.665733... Val Loss: 0.654801
Epoch: 2/6... Step: 600... Loss: 0.593303... Val Loss: 0.643527
Epoch: 2/6... Step: 700... Loss: 0.680365... Val Loss: 0.691243
Epoch: 2/6... Step: 800... Loss: 0.731569... Val Loss: 0.688889
Epoch: 3/6... Step: 900... Loss: 0.607726... Val Loss: 0.585775
Epoch: 3/6... Step: 1000... Loss: 0.508429... Val Loss: 0.583774
Epoch: 3/6... Step: 1100... Loss: 0.448282... Val Loss: 0.462154
Epoch: 3/6... Step: 1200... Loss: 0.347062... Val Loss: 0.447624
Epoch: 4/6... Step: 1300... Loss: 0.340563... Val Loss: 0.465717
Epoch: 4/6... Step: 1400... Loss: 0.423231... Val Loss: 0.487504
Epoch: 4/6... Step: 1500... Loss: 0.396020... Val Loss: 0.473595
Epoch: 4/6... Step: 1600... Loss: 

## Loading the best model

In [0]:
net = load_checkpoint('Best_RNN.pth')
print(net)

### Inference on a test review

In [0]:
from string import punctuation
def preprocess_review(review, sequence_length=200): 
  lower_review = review.lower()
  print('Lower case: ', lower_review)
  clean_review = ''.join([c for c in lower_review if c not in punctuation])
  print('Clean : ', clean_review)
  encoded_review = [[vocab_to_int[word] for word in clean_review.split()]]
  print('Encoded case: ', encoded_review)
  padded_review = pad_features(encoded_review, sequence_length)
  print('Padded case: ', padded_review)

  return torch.from_numpy(padded_review)

def predict(net, test_review, sequence_length=200, include_neutral=False):
    ''' Prints out whether a given review is predicted to be 
        positive or negative in sentiment, using a trained model.
        
        params:
        net - A trained net 
        test_review - a review made of normal text and punctuation
        sequence_length - the padded length of a review
        '''
    net.eval()
    review = preprocess_review(test_review, sequence_length)
    if(train_on_gpu):
        review = review.cuda()

    batch_size = review.size(0)
    h = net.init_hidden(batch_size)

    # get predicted outputs
    output, h = net(review, h)    
    prob = output.squeeze()
    print('Probability: ', prob.item()) 

    # these two parameters control what's considered a neutral comment. 
    upper_limit = 0.6
    lower_limit = 0.4

    if (include_neutral == False) or prob.gt(upper_limit) or prob.lt(lower_limit): 
      pred = torch.round(prob)  # round to predicted class (0 or 1)
      print('\n\n {} comment detected!'.format('Negative' if pred.eq(0) else 'Positive' ))
    else: 
      print('\n\n Neutral comment detected!')

In [0]:
seq_length=200

# positive test review
test_review_pos = 'This movie had the best acting and the dialogue was so good. I loved it.'

# negative test review
test_review_neg = 'The worst possible movie I have seen; acting was terrible and I want my money back. This movie had bad acting and the dialogue was slow.'

# try negative and positive reviews!
predict(net, test_review_pos, seq_length)